In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import normalize

from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import recall_score, f1_score

carac = pd.read_csv("C:/Users/darla/Desktop/Logements data/Base d'apprentissage/carac.csv",sep=';')
lieux = pd.read_csv("C:/Users/darla/Desktop/Logements data/Base d'apprentissage/lieux.csv",sep=';')
veh = pd.read_csv("C:/Users/darla/Desktop/Logements data/Base d'apprentissage/veh.csv",sep=';')
vict = pd.read_csv("C:/Users/darla/Desktop/Logements data/Base d'apprentissage/vict.csv",sep=';')

victime = vict.merge(veh,on=['Num_Acc','num_veh'])
accident = carac.merge(lieux,on = 'Num_Acc')
victime = victime.merge(accident,on='Num_Acc')

nans = ['v1','v2','lartpc','larrout','locp','etatp','actp','voie','pr1','pr','place']

victime = victime.drop(columns = nans)

victime = victime.dropna()

victime.corr()
victime.var()

victime = victime.drop(columns=['an'])

hrmn=pd.cut(victime['hrmn'],24,labels=[str(i) for i in range(0,24)])
victime['hrmn']=hrmn.values

# On extrait du tableau la latitude et la longitude

X_lat = victime['lat']
X_long = victime['long']

# On définit tous nos points à classifier

X_cluster = np.array((list(zip(X_lat, X_long))))

# Kmeans nous donne pour chaque point la catégorie associée

clustering = KMeans(n_clusters=15, random_state=0)
clustering.fit(X_cluster)

# Enfin on ajoute les catégories dans la base d'entraînement

geo = pd.Series(clustering.labels_)
victime['geo'] = geo

y = victime['grav']

features = ['catu','sexe','trajet','secu',
            'catv','an_nais','mois',
            'occutc','obs','obsm','choc','manv',
            'lum','agg','int','atm','col','gps',
            'catr','circ','vosp','prof','plan',
            'surf','infra','situ','hrmn','geo']

X_train = pd.get_dummies(victime[features].astype(str))

# On redécoupe la base en train/test

X_train, X_test, y_train, y_test = train_test_split(X_train,y)


# On crée le modèle :

model_boosting = GradientBoostingClassifier(loss="deviance",
    learning_rate=0.2,
    max_depth=5,
    max_features="sqrt",
    subsample=0.95,
    n_estimators=200)

# L'entraînement débute :

model_boosting.fit(X_train, y_train)

# On calcul les prédictions
predictions_test_xgb = model_boosting.predict(X_test)
predictions_train_xgb = model_boosting.predict(X_train)

# On affiche les résultats :

train_acc = accuracy_score(y_train, predictions_train_xgb)
print(train_acc)

test_acc = accuracy_score(y_test, predictions_test_xgb)
print(test_acc)

C:\Users\darla\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\darla\AppData\Local\Temp/ipykernel_17132/2088869281.py:31: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  victime.var()


0.7006002400960384
0.6564000548772122
